<a href="https://colab.research.google.com/github/ykitaguchi77/manipulate_CSV/blob/master/Extract_specific_disease_files_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 特定の疾患の患者の画像ファイルを、眼位写真Newから抜き出す

In [ ]:
import pandas as pd

# shift_jis での読み込みを試みる
try:
    df_all = pd.read_csv("F:\●DiseaseInfo_all_connected_20230907.csv", encoding="shift_jis", header=None)
except UnicodeDecodeError:
    # cp932 での読み込みを試みる
    df_all = pd.read_csv("F:\●DiseaseInfo_all_connected_20230907.csv", encoding="cp932", header=None)

# 「眼窩腫瘍」あるいは「涙腺腫瘍」が含まれている行を抜き出す
filtered_df = df_all[df_all.apply(lambda row: row.astype(str).str.contains('眼窩腫瘍|涙腺腫瘍').any(), axis=1)]

filtered_df

filtered_df.to_csv(r"D:\ダウンロード\filtered_diseases_眼窩腫瘍_涙腺腫瘍.csv", index=False, header=0, encoding="shift_jis")



In [ ]:
# C列 (2番目のカラム) のデータをリストに変換
patient_number_list = filtered_df.iloc[:, 2].tolist()

# patient_number_list = [
#     465, 1020, 1304, 1396, 1773, 1995, 2941, 3176, 3600, 3684, 3759, 3809,
#     4125, 4588, 4939, 5125, 5203, 5299, 5890, 6515, 7049, 7765, 24, 36, 1309,
#     2765, 3079, 3146, 3428, 3496, 3832, 4186, 5491, 5541, 5582, 7161, 7397,
#     7870, 7987, 1041, 1259, 1483, 1508, 2736, 4388, 4874, 5880, 6269, 6320,
#     6654, 6663, 6965, 7025, 7507, 8105, 17, 19, 57, 122, 129, 155, 180, 374,
#     385, 425, 426, 429, 466, 489, 507, 533, 534, 555, 611, 718, 724, 782, 787,
#     818, 876, 893, 950, 957, 965, 977, 983, 1001, 1037, 1088, 1231, 1270, 1284,
#     1313, 1333, 1336, 1347, 1357, 1370, 1378, 1385, 1429, 1432, 1438, 1446, 1453,
#     1457, 1501, 1503, 1524, 1525, 1528, 1553, 1597, 1664, 1682, 1729, 1764, 1743,
#     1786, 1815, 1841, 1910, 1913, 1931, 2007, 2042, 2148, 2182, 2189, 2203, 2222,
#     2231, 2274, 2307, 2321, 2370, 2407, 2414, 2436, 2508, 2510, 2558, 2623, 2630,
#     2644, 2658, 2664, 2668, 2670, 2684, 2689, 2709, 2726, 2762, 2794, 2801, 2803,
#     2837, 2851, 2853, 2885, 2900, 2960, 2967, 2974, 3046, 3056, 3077, 3128, 3144,
#     3162, 3164, 3167, 3209, 3217, 3237, 3264, 3276, 3331, 3352, 3412, 3444, 3472,
#     3484, 3502, 3528, 3532, 3543, 3567, 3568, 3609, 3646, 3667, 3695, 3714, 3748,
#     3752, 3783, 3802, 3843, 3875, 3918, 3929, 3942, 3976, 4008, 4030, 4069, 4206,
#     4209, 4233, 4263, 4272, 4292, 4331, 4332, 4359, 4382, 4344, 4465, 4480, 4485,
#     4486, 4540, 4553, 4577, 4606, 4616, 4618, 4619, 4628, 4652, 4681, 4710, 4713,
#     4759, 4805, 4813, 4822, 4881, 4921, 4930, 4958, 4993, 4997, 5015, 5021, 5051,
#     5069, 5070, 5095, 5310, 5237, 5260, 5273, 5290, 5297, 5323, 5333, 5335, 5337,
#     5378, 5397, 5426, 5540, 5543, 5546, 5550, 5551, 5571, 5586, 5607, 5608, 5646,
#     5647, 5673, 5681, 5758, 5763, 5768, 5820, 5838, 5865, 5888, 5901, 5906, 5913,
#     5965, 5989, 6003, 6074, 6106, 6147, 6150, 6155, 6163, 6277, 6283, 6297, 6298,
#     6355, 6432, 6478, 6486, 6489, 6502, 6513, 6518, 6553, 6555, 6579, 6599, 6643,
#     6662, 6707, 6725, 6764, 6788, 6806, 6855, 6956, 6989, 7041, 7044, 7077, 7104,
#     7166, 7277, 7283, 7291, 7295, 7302, 7346, 7349, 7403, 7419, 7428, 7436, 7443,
#     7449, 7455, 7527, 7534, 7552, 7644, 7669, 7681, 7709, 7745, 7763, 7781, 7798,
#     7800, 7867, 7874, 7879, 7900, 7936, 8079, 8168, 8184, 8198
# ]

patient_number_list


In [ ]:
"""
特定の数字リストに基づいて眼位写真NEW内のファイルをフィルタリングし、マッチするファイルのフルパスをリストに保存

"""

import os
import re

# 1. Create an empty list
resulting_files = []

# 2. Iterate over all files in the specified directory
# Note: This code cannot be run in this environment, but is provided for reference.
for filename in os.listdir("F:\\眼位写真NEW"):
    # 3. Extract the number from the basename
    base_name = os.path.basename(filename)
    match = re.match(r"^\d+", base_name)
    if match:
        number = int(match.group(0))
        # 4. Check if the number is in the specified list
        if number in patient_number_list:
            resulting_files.append(os.path.join("F:\\眼位写真NEW", filename))

# Output the resulting_files list (commented out since we cannot run the os.listdir in this environment)
resulting_files


In [ ]:
def extract_dates_and_years(file_paths):
    # 日付を抜き出す
    dates = []
    for path in file_paths:
        date_match = re.search(r'(\d{8})', path)
        if date_match:
            dates.append(date_match.group(1))

    # 日付から年を抜き出す
    years = [date[:4] for date in dates]

    return dates, years

def filter_files_by_year(file_paths, start_year):
    _, years = extract_dates_and_years(file_paths)

    # Filtering files based on the year
    filtered_files = [file for file, year in zip(file_paths, years) if int(year) >= start_year]

    return filtered_files

# Filtering the resulting_files list for files from 2017 onwards
filtered_files_2017_onwards = filter_files_by_year(resulting_files, 2017)
filtered_files_2017_onwards


In [75]:
len(filtered_files_2017_onwards)

1430

In [77]:
from tqdm import tqdm
import shutil

destination_folder = "F:\\眼位写真\\眼位写真_甲状腺眼症"

# If the folder already exists, delete it
if os.path.exists(destination_folder):
    shutil.rmtree(destination_folder)
os.makedirs(destination_folder)

# Copy files to the destination folder with tqdm progress bar
for file_path in tqdm(filtered_files_2017_onwards, desc="Copying files"):
    shutil.copy(file_path, destination_folder)

Copying files: 100%|███████████████████████████████████████████████████████████████| 1430/1430 [04:00<00:00,  5.96it/s]


In [ ]:
data = []

for path in filtered_files_2017_onwards:
    matches = re.findall(r'(\d+)-(\d+)-\d+-\d+_(\w+)\.', path)
    if matches:
        row = []
        for match in matches[0]:
            if match.isdigit():
                row.append(int(match))
            else:
                row.append(match)
        data.append(row)

df = pd.DataFrame(data, columns=['Patient_num', 'date', 'Hash'])
df[['Patient_id', 'Hertel_R', 'Hertel_L']] = None #空の行を追加
df = df[['Patient_id', 'Patient_num', 'date', 'Hash', 'Hertel_R', 'Hertel_L']] #並べ替え


df

In [ ]:
# filtered_dfの2列目とdfのPatient_numが一致する場合、
# filtered_dfの0列目の内容をdfのPatient_id列に書き込む
for index, row in df.iterrows():
    matching_rows = df_all[df_all.iloc[:, 2] == row['Patient_num']]
    if not matching_rows.empty:
        df.at[index, 'Patient_id'] = "{:08}".format(int(matching_rows.iloc[0, 0]))

# Drop the 'Hash' column
try:
    df = df.drop(columns=['Hash'])
except:
    pass

# Drop duplicate rows based on 'Patient_num'
df.drop_duplicates(subset=['Patient_num', 'date'], inplace=True)

# Sort the dataframe by 'date'
df = df.sort_values(by='date')

# Sort the dataframe by 'Patient_num'
df = df.sort_values(by='Patient_num')



# Reset the index
df = df.reset_index(drop=True)



df


In [104]:
df.to_csv(r"D:\ダウンロード\filtered_diseases_眼窩涙腺腫瘍_Hertel.csv", index=False, header=True, encoding="shift_jis")


In [ ]:
df